# **Ejercicio 3: Activo libre de riesgo y Capital Market Line (CML)**

#### Objetivo: 
Incorporar un activo libre de riesgo (por ejemplo, un Treasury con un 3\% anual) y construir la Capital Market Line (CML), encontrando la cartera tangente que maximiza el ratio de Sharpe.


### **1. Descargar y preparar los datos**
- Obtener precios históricos de varios activos (ejemplo: AAPL, MSFT, TSLA, AMZN).
- Convertir los precios en retornos logarítmicos diarios.
- Calcular:
 Vector de retornos medios diarios: 
 $\mu$,
 Matriz de covarianzas: $\sigma$.


### **2. Simulación de carteras riesgosas (sin activo libre de riesgo)**

Generar muchos vectores de pesos aleatorios $w$ que sumen 1.
Para cada vector calcular:
    $$
    \mu_p = w^T \mu, \quad
    \sigma_p = \sqrt{w^T \Sigma w}, \quad
    S_p = \frac{\mu_p}{\sigma_p}.
    $$
Identificar la cartera con el mayor Sharpe ratio $S_p$ (cartera tangente).


### **3. Introducir el activo libre de riesgo**

Supongamos un rendimiento libre de riesgo de $r_f = 0.03$.
El Sharpe ratio ajustado es:
    $
    S^* = \frac{\mu_p - r_f}{\sigma_p}.
    $
La ecuación de la Capital Market Line es:
    $
    E[R_c] = r_f + S^* \cdot \sigma_c.
    $


### **4. Generar combinaciones**

Crear una malla de volatilidades $\sigma_c$ (ej. 0\%--30\%).
Para cada $\sigma_c$, calcular $E[R_c]$ mediante la ecuación de la CML.


### **5. Visualización**

Graficar las carteras simuladas en el plano $(\sigma_p, \mu_p)$.
 Dibujar la recta de la CML.
 Marcar la cartera tangente con un símbolo destacado.


### **6. Interpretación**
La CML representa la mejor combinación posible entre riesgo y retorno.
Inversores conservadores tenderán a asignar mayor peso al activo libre de riesgo.
 Inversores arriesgados pueden incluso apalancarse en la cartera tangente.

In [24]:
# Importamos las librerias
import yfinance as yf
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10

In [25]:
# 1️⃣ Descargar datos
tickers = ["C", "UPS", "PFE", "COP"]
start = '2020-01-01'
end = '2025-09-30'
data = yf.download(tickers, start=start, end=end)['Close']

[*********************100%***********************]  4 of 4 completed


In [26]:
# Crear figura
p = figure(
    title='Activos Financieros - Precios de Cierre',
    x_axis_label='Fecha',
    y_axis_label='Precio de Cierre',
    x_axis_type='datetime',
    tools='pan,wheel_zoom,box_zoom,reset'
)

# Graficar cada ticker
colors = Category10[10]
for i, ticker in enumerate(tickers):
    source = ColumnDataSource(data={
        'x': data.index,
        'y': data[ticker],
        'ticker': [ticker]*len(data)
    })
    
    p.line('x', 'y', source=source, line_width=2, color=colors[i], legend_label=ticker)
    p.circle('x', 'y', source=source, size=1.5, color=colors[i], alpha=0.6)

# Añadir HoverTool
hover = HoverTool(
    tooltips=[
        ("Ticker", "@ticker"),
        ("Fecha", "@x{%F}"),
        ("Cierre", "@y{0.2f}")
    ],
    formatters={'@x': 'datetime'},
    mode='vline'
)
p.add_tools(hover)

# Leyenda interactiva
p.legend.location = "top_left"
p.legend.click_policy = "hide"

# Mostrar gráfico
show(p)


In [27]:
# Graficamos los mismos datos en una grafica logaritmica

datalog = np.log(data)
print (data)

# Crear figura
p = figure(
    title='Activos Financieros - Precios de Cierre Logaritmicos',
    x_axis_label='Fecha',
    y_axis_label='Precio de Cierre',
    x_axis_type='datetime',
    tools='pan,wheel_zoom,box_zoom,reset'
)

# Graficar cada ticker
colors = Category10[10]
for i, ticker in enumerate(tickers):
    source = ColumnDataSource(data={
        'x': data.index,
        'y': datalog[ticker],
        'ticker': [ticker]*len(datalog)
    })
    
    p.line('x', 'y', source=source, line_width=2, color=colors[i], legend_label=ticker)
    p.circle('x', 'y', source=source, size=1.5, color=colors[i], alpha=0.6)

# Añadir HoverTool
hover = HoverTool(
    tooltips=[
        ("Ticker", "@ticker"),
        ("Fecha", "@x{%F}"),
        ("Cierre", "@y{0.2f}")
    ],
    formatters={'@x': 'datetime'},
    mode='vline'
)
p.add_tools(hover)

# Leyenda interactiva
p.legend.location = "top_left"
p.legend.click_policy = "hide"

# Mostrar gráfico
show(p)


Ticker               C        COP        PFE        UPS
Date                                                   
2020-01-02   65.700394  53.051498  28.447659  93.547058
2020-01-03   64.462906  53.245995  28.295033  93.490982
2020-01-06   64.260696  53.878155  28.258696  93.074478
2020-01-07   63.702606  53.878155  28.164207  92.914284
2020-01-08   64.187904  52.630066  28.389519  93.442947
...                ...        ...        ...        ...
2025-09-18  102.410004  93.480003  24.150000  85.050003
2025-09-19  102.680000  91.919998  24.030001  84.059998
2025-09-22  103.489998  92.010002  24.040001  84.230003
2025-09-23  103.000000  93.470001  24.129999  84.410004
2025-09-24  101.650002  95.629997  24.090000  83.900002

[1440 rows x 4 columns]


In [28]:
# Calculamos los retornos logaritmicos
diff = data.diff()
ret_log = np.log (data/data.shift(1))

# Calculamos la media y la covarianza
mean_returns = ret_log.mean()
Sigma = ret_log.cov().values
mu = ret_log.mean().values

In [33]:
# Generamos multiples vectores de pesos aleatorios
n_assets = len(tickers)         # número de activos
n_portfolios = 100000 # número de carteras a generar

# Generar vectores de pesos aleatorios
weights = np.random.dirichlet(np.ones(n_assets), size=n_portfolios)

print(weights.shape)  # (10000, 4)
print("Ejemplo de pesos:", weights[0], "suma =", weights[0].sum())

# --- 3. Calcular métricas de cada cartera ---
# Retornos esperados
port_returns = weights.dot(mu)

# Varianzas y volatilidades
port_vars = np.einsum('ij,jk,ik->i', weights, Sigma, weights)
port_vols = np.sqrt(port_vars)

# Sharpe ratio (rf = 0)
sharpe_ratios = port_returns / (port_vols + 1e-12)  # evitar división por cero

# --- 4. Encontrar cartera tangente (máximo Sharpe) ---
idx_max_sharpe = np.argmax(sharpe_ratios)
w_tangente = weights[idx_max_sharpe]

print("Sharpe máximo:", sharpe_ratios[idx_max_sharpe])
print("Pesos de la cartera tangente:")
for t, w in zip(tickers, w_tangente):
    print(f" - {t}: {w:.3f}")
    
    

if sharpe_ratios[idx_max_sharpe] < 1:
    print("La cartera tangente tiene un Sharpe bajo: poco eficiente en relación riesgo-retorno.")
elif best_sharpe < 2:
    print("La cartera tangente tiene un Sharpe aceptable: riesgo-retorno moderado.")
else:
    print("La cartera tangente tiene un Sharpe alto: muy eficiente en relación riesgo-retorno.")
    
print ("Además, La cartera tangente es la combinación de activos que ofrece la mejor compensación riesgo-retorno." 
       "Por lo tanto, teniendo en cuenta los activos financieros seleccionados, el sharpe máximo es", 
       sharpe_ratios[idx_max_sharpe], "Y este valor mide la rentabilidad ajustada por riesgo" )



(100000, 4)
Ejemplo de pesos: [0.58059437 0.38166049 0.01509311 0.02265203] suma = 0.9999999999999998
Sharpe máximo: 0.015488094185500305
Pesos de la cartera tangente:
 - C: 0.376
 - UPS: 0.619
 - PFE: 0.004
 - COP: 0.001
La cartera tangente tiene un Sharpe bajo: poco eficiente en relación riesgo-retorno.
Además, La cartera tangente es la combinación de activos que ofrece la mejor compensación riesgo-retorno.Por lo tanto, teniendo en cuenta los activos financieros seleccionados, el sharpe máximo es 0.015488094185500305 Y este valor mide la rentabilidad ajustada por riesgo
